# W08: Rutherford Scattering

* Last revision: 20-Oct-2021 by Heiko Hergert (hergert@frib.msu.edu)

## Simulation

In [1]:
%matplotlib inline

In [2]:
from time import sleep

import numpy as np
from simulator import Nucleus, Simulator

from matplotlib import cm
from vpython import *


<IPython.core.display.Javascript object>

We should choose natural units for our simulation, so that we do not have to worry about roundoff errors when dealing with numbers that cover vastly different orders of magnitudes (e.g., $G\sim10^{-10}$, $M \sim 10^{30}$, etc.). **Experiment with the length scale to achieve good visualization results!**

In [3]:
# set up parameters
alpha = 1/137.

In [16]:
# set up the VPython scene
scene = canvas(title='Rutherford Scattering',
            width=600, height=400,
            center=vector(0,0,0), background=color.white)

# For some reason, the creators thought it would be a good idea to have y be the
# upward direction. We'll change that to the z direction.
scene.forward = vector(1,0,0)
scene.up = vector(0,0,1)



# Define and initiate the simulated objects - remember to translate to natural units!
gold     = Nucleus(alpha,
                      mass = 19700000,
                      charge = 79,
                      pos=vector(0,0,0),
                      velocity=vector(0,0,0),
                      color=vector(240,215,0)/255.,
                      radius=0.2)

objects = [gold]

cols = cm.get_cmap('inferno', 10).colors[1:]
#cols = cm.get_cmap('Blues_r')

vec_cols = []
for c in cols:
    vec_cols.append(vector(c[0],c[1],c[2]))
    
# blues = cm.get_cmap('Blues_r')

for i, b in enumerate([0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0, 2]):
#for i, b in enumerate([0.1]):
    objects.append(
        Nucleus(alpha, 
              mass = 4,
              charge = 2,
              pos=vector(-10,b,0),
              velocity=vector(1,0,0), 
              color=vec_cols[i],
              radius=0.005)

    )


# Create the list of objects... 

# ... and initiate the simulator with time step dt.
sim = Simulator(objects, alpha, dt=0.01)

# Choose the time span of the simulation.
tmax = 20

# Create a VPython graph object for the potential energy.
vgraph = graph(x=0, y=0,width=400,height=400,\
              title = 'Potential Energy', \
              xtitle = 't', ytitle = 'V [fm<sup>-1</sup>]', \
              foreground = color.black, background =color.white, \
              xmax = tmax, xmin = 0)

# All subsequently defined VPython curve objects are children of the
# same graph until the next graph object is created.
vcurves=[ ]
for obj in objects[1:]:
    vcurves.append(gcurve(color=obj.color))

# Graph for the effective potential
veffgraph = graph(x=0, y=0,width=400,height=400,\
              title = 'Effective Potential Energy', \
              xtitle = 't', ytitle = 'V<sub>eff</sub> [fm<sup>-1</sup>]', \
              foreground = color.black, background =color.white, \
              xmax = tmax, xmin = 0)

veffcurves=[ ]
for obj in objects[1:]:
    veffcurves.append(gcurve(color=obj.color))


# Same graph for the total and radial kinetic energies...
tgraph = graph(x=0, y=0,width=400,height=400,\
                  title = 'Kinetic Energy (Radial + Angular)', \
                  xtitle = 't', ytitle = 'T [fm<sup>-1</sup>]', \
              foreground = color.black, background =color.white, \
              xmax = tmax, xmin = 0)

tcurves=[ ]
for obj in objects[1:]:
    tcurves.append(gcurve(color=obj.color))

tradgraph = graph(x=0, y=0,width=400,height=400,\
                  title = 'Radial Kinetic Energy', \
                  xtitle = 't', ytitle = 'T<sub>r</sub> [fm<sup>-1</sup>]', \
              foreground = color.black, background =color.white, \
              xmax = tmax, xmin = 0)

tradcurves=[ ]
for obj in objects[1:]:
    tradcurves.append(gcurve(color=obj.color))


# Initialize step counter...
steps = 0

sim.transform_to_com()

# ... and start the simulation.
while steps * sim.dt < tmax:

    # VPython animation rate.
    rate(100)
    
    # Take a time step.
    sim.update_verlet()

    # Update energy graphs.
    for i, obj in enumerate(objects[1:]):
        tcurves[i].plot(steps*sim.dt, obj.T)
        vcurves[i].plot(steps*sim.dt, obj.V)
        tradcurves[i].plot(steps*sim.dt, obj.Trad)
        veffcurves[i].plot(steps*sim.dt, obj.Veff)
        
    steps+=1


<IPython.core.display.Javascript object>

<-1.73205, 0, 0>
<1.73205, 0, 0>
<1, 0, 0>
<0, 0, 1>


You can see how the radial kinetic energy $\frac{1}{2}m\dot{r}^2$ now actually goes to zero as we reach the pericenter, the projectile's point of closest approach to the center of the scattering potential. At the same time, $V_\text{eff}$ has a maximum, and it's clear that we must have $E=V_\text